# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saipan,15.1355,145.7010,29.39,100,100,8.75,MP,2023-04-22 03:28:07
1,1,College,64.8569,-147.8028,6.97,49,100,1.54,US,2023-04-22 03:29:31
2,2,Port-Aux-Francais,-49.3500,70.2167,4.88,71,86,7.99,TF,2023-04-22 03:29:31
3,3,Polyarnyy,69.1989,33.4478,-3.71,78,96,6.91,RU,2023-04-22 03:29:32
4,4,Burang,30.2500,81.1667,-5.85,79,48,1.70,CN,2023-04-22 03:29:32


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
locations = city_data_df.hvplot.points(
    "Lng",
     "Lat", 
     geo = True, 
     tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"   )

# Display the map
locations

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Humidity"] < 70)]

# Drop any rows with null values
narrowed_city_data_df = narrowed_city_data_df.dropna(how='any')

# Display sample data
narrowed_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,Kingscliff,-28.2667,153.5667,24.95,57,75,8.75,AU,2023-04-22 03:29:41
25,25,Geraldton,-28.7667,114.6000,29.23,23,85,8.23,AU,2023-04-22 03:29:45
34,34,Sabha,27.0377,14.4283,23.41,15,39,3.41,LY,2023-04-22 03:29:51
36,36,Loei,17.4167,101.5000,28.95,30,0,1.09,TH,2023-04-22 03:29:53
94,94,Tidjikja,18.5564,-11.4271,26.39,20,97,4.90,MR,2023-04-22 03:30:38


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_data_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,18,Kingscliff,-28.2667,153.5667,24.95,57,75,8.75,AU,2023-04-22 03:29:41,
25,25,Geraldton,-28.7667,114.6000,29.23,23,85,8.23,AU,2023-04-22 03:29:45,
34,34,Sabha,27.0377,14.4283,23.41,15,39,3.41,LY,2023-04-22 03:29:51,
36,36,Loei,17.4167,101.5000,28.95,30,0,1.09,TH,2023-04-22 03:29:53,
94,94,Tidjikja,18.5564,-11.4271,26.39,20,97,4.90,MR,2023-04-22 03:30:38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {
    "categories": categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kingscliff - nearest hotel: No hotel found
Geraldton - nearest hotel: No hotel found
Sabha - nearest hotel: No hotel found
Loei - nearest hotel: No hotel found
Tidjikja - nearest hotel: No hotel found
Santa Ana - nearest hotel: No hotel found
Margaret River - nearest hotel: No hotel found
Al Majaridah - nearest hotel: Хотел Вила Анастасија
Bengkulu - nearest hotel: No hotel found
Agadez - nearest hotel: No hotel found
San Patricio - nearest hotel: No hotel found
Maumere - nearest hotel: No hotel found
Pimentel - nearest hotel: No hotel found
Lakatoro - nearest hotel: No hotel found
Alofi - nearest hotel: No hotel found
Manadhoo - nearest hotel: No hotel found
Alice Springs - nearest hotel: No hotel found
Charters Towers - nearest hotel: No hotel found
Gwadar - nearest hotel: No hotel found
Dampit - nearest hotel: No hotel found
Cockburn Town - nearest hotel: No hotel found
Port Augusta - nearest hotel: No hotel found
Moriki - nearest hotel: No hotel found
Bubaque 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,18,Kingscliff,-28.2667,153.5667,24.95,57,75,8.75,AU,2023-04-22 03:29:41,No hotel found
25,25,Geraldton,-28.7667,114.6000,29.23,23,85,8.23,AU,2023-04-22 03:29:45,No hotel found
34,34,Sabha,27.0377,14.4283,23.41,15,39,3.41,LY,2023-04-22 03:29:51,No hotel found
36,36,Loei,17.4167,101.5000,28.95,30,0,1.09,TH,2023-04-22 03:29:53,No hotel found
94,94,Tidjikja,18.5564,-11.4271,26.39,20,97,4.90,MR,2023-04-22 03:30:38,No hotel found
101,101,Santa Ana,33.7456,-117.8678,26.98,34,20,3.60,US,2023-04-22 03:30:43,No hotel found
106,106,Margaret River,-33.9500,115.0667,23.74,37,0,2.62,AU,2023-04-22 03:30:46,No hotel found
121,121,Al Majaridah,19.1236,41.9111,28.69,48,3,1.37,SA,2023-04-22 03:30:55,Хотел Вила Анастасија
127,127,Bengkulu,-3.8004,102.2655,28.85,69,47,1.30,ID,2023-04-22 03:29:18,No hotel found
145,145,Agadez,19.7500,10.2500,27.09,14,29,5.56,NE,2023-04-22 03:31:14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
     "Lat", 
     geo = True, 
     tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"   )


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)